In [1]:
import io
import os
import math
import time
import json
import torch
import hashlib
import tempfile
import bittensor as bt
from types import SimpleNamespace
from transformers import GPT2Config, GPT2LMHeadModel
from transformers import LlamaForCausalLM, LlamaConfig, LlamaTokenizer
from typing import Dict, List, Optional, Tuple
import io
import os
import math
import time
import boto3
import torch
import wandb
import typer
import argparse
import tempfile
import bittensor as bt
import numpy as np
from tqdm import tqdm
import torch.optim as optim
from dotenv import dotenv_values
from types import SimpleNamespace
from transformers import AutoTokenizer
from transformers import GPT2Config, GPT2LMHeadModel
from typing import Dict, List, Optional, Tuple

# Instantiate my S3 client.
env_config = {**dotenv_values(".env"), **os.environ}
AWS_ACCESS_KEY_ID = env_config.get('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = env_config.get('AWS_SECRET_ACCESS_KEY')
client: boto3.client = boto3.client(
    's3',
    region_name='us-east-1',
    aws_access_key_id = AWS_ACCESS_KEY_ID,
    aws_secret_access_key = AWS_SECRET_ACCESS_KEY
)
bucket = 'decis'

In [3]:
from common import get_latest_metadata

In [5]:
latest_master_meta = get_latest_metadata( 0, bt.subtensor('test').metagraph(212), bt.subtensor('test'), CLIENT = client )


In [6]:
latest_master_meta

namespace(last_modified=1726004446,
          blocks_since_modified=32,
          bucket='decis',
          filename='model-5F4UUMWF41GsLFvwVpigmoxTKVngDP6C7utvECgtQB83U3fJ-1726004438.pt',
          metadata_filename='model-5F4UUMWF41GsLFvwVpigmoxTKVngDP6C7utvECgtQB83U3fJ-1726004438_metadata.json',
          sequence_length=2048,
          tokenizer_name='gpt2',
          deltas=[{'last_modified': 1726004389,
                   'blocks_since_modified': 2,
                   'bucket': 'decis',
                   'filename': 'model-5EkVGe9zdK4D1rdPD2qutAbU1HGNcjSt8ibsMQcWnPLNcsxn-1726004382.pt',
                   'metadata_filename': 'model-5EkVGe9zdK4D1rdPD2qutAbU1HGNcjSt8ibsMQcWnPLNcsxn-1726004382_metadata.json',
                   'master_hash': 'd15e331787bf1f251183313ada6b7a4a645e38959b468b6ae80957a8bd60ac81',
                   'model_type': 'llama',
                   'model_config': {'vocab_size': 50257,
                    'max_position_embeddings': 2048,
                    'h

In [6]:

response = client.list_objects_v2( Bucket = bucket )
file_names = [content['Key'] for content in response.get('Contents', [])]
print (file_names)

['model-5D2uD6jYfLvpMPsngA3n4VAqxxmXSPtuku1TaXQnPdH3xohZ.pt', 'model-5D2uD6jYfLvpMPsngA3n4VAqxxmXSPtuku1TaXQnPdH3xohZ_metadata.json', 'model-5DHYeSMbG3n31Ev9e9BNZzBiWnSd9bkp5s8CdC3RR5aMUpAh.pt', 'model-5DHYeSMbG3n31Ev9e9BNZzBiWnSd9bkp5s8CdC3RR5aMUpAh_metadata.json', 'model-5EkVGe9zdK4D1rdPD2qutAbU1HGNcjSt8ibsMQcWnPLNcsxn.pt', 'model-5EkVGe9zdK4D1rdPD2qutAbU1HGNcjSt8ibsMQcWnPLNcsxn_metadata.json', 'model-5F4UUMWF41GsLFvwVpigmoxTKVngDP6C7utvECgtQB83U3fJ.pt', 'model-5F4UUMWF41GsLFvwVpigmoxTKVngDP6C7utvECgtQB83U3fJ_metadata.json', 'model-5GQw7vvU1tzQ8XCVE1FRf3As9MmTZasPZUoiSgDmnnEj9jaP.pt', 'model-5GQw7vvU1tzQ8XCVE1FRf3As9MmTZasPZUoiSgDmnnEj9jaP_metadata.json']


In [8]:
model = GPT2LMHeadModel( config = GPT2Config(
    output_hidden_states = False, 
    n_positions = 1024
))

wallet = bt.wallet('Alice', 'Alice')

In [17]:
def get_latest_metadata_file( hotkey ) -> str:
    response = client.list_objects_v2( Bucket = bucket )
    file_names = [content['Key'] for content in response.get('Contents', [])]
    max_block = -1
    latest_file = None
    for file_name in file_names:
        if file_name.startswith(f'gradients-{wallet.hotkey.ss58_address}-') and file_name.endswith('.pt'):
            try:
                block = int(file_name.split('-')[-1].split('.')[0])
                if block > max_block:
                    max_block = block
                    latest_file = file_name
            except ValueError:
                continue
    print(f"Latest file with the largest block: {latest_file}")
    return latest_file

get_latest_metadata_file( wallet.hotkey.ss58_address )


Latest file with the largest block: gradients-5F4UUMWF41GsLFvwVpigmoxTKVngDP6C7utvECgtQB83U3fJ-11.pt


'gradients-5F4UUMWF41GsLFvwVpigmoxTKVngDP6C7utvECgtQB83U3fJ-11.pt'

In [16]:
block = 11
filename = f'gradients-{wallet.hotkey.ss58_address}-{block}.pt'  # Filename for the gradients
gradients = {name: param.data for name, param in model.named_parameters() if param.grad is not None}
with io.BytesIO() as gradients_buffer:
    torch.save(gradients, gradients_buffer)  # Save the gradients to the buffer
    gradients_buffer.seek(0)  # Reset the buffer's position to the beginning
    client.upload_fileobj(gradients_buffer, bucket, filename)  # Upload the gradients buffer to the storage service
    client.put_object_acl(
        Bucket=bucket,
        Key=filename,
        GrantRead='uri="http://acs.amazonaws.com/groups/global/AllUsers"',
        GrantReadACP='uri="http://acs.amazonaws.com/groups/global/AllUsers"'
    )


In [8]:
eval_pages = SubsetFineWebEdu2Loader.next_pages( offset = 10000003000, n_pages = 10, seed = 9 )
print ([e[1] for e in eval_pages])

[28502488, 22687085, 57214137, 41008498, 28242177, 9211837, 27743392, 14221368, 15647757, 57330970]


In [7]:
eval_pages = SubsetFineWebEdu2Loader.next_pages( offset = 10000003001, n_pages = 10, seed = 9 )
print ([e[1] for e in eval_pages])

[22687085, 57214137, 41008498, 28242177, 9211837, 27743392, 14221368, 15647757, 57330970, 47115201]


In [2]:

import random
from transformers import AutoTokenizer
from dataset import SubsetFineWebEdu2Loader
tokenizer: AutoTokenizer = AutoTokenizer.from_pretrained( 'gpt2', verbose=False, clean_up_tokenization_spaces=True )
tokenizer.pad_token = tokenizer.eos_token    


eval_pages = SubsetFineWebEdu2Loader.next_pages( offset = 100000, n_pages = 10, seed = 10 )
            
# training_pages = [ random.choice( eval_pages ) ]
# print ('training page', training_pages )

# dataset = SubsetFineWebEdu2Loader(
#     batch_size = 10,
#     sequence_length = 2048,
#     pages_info = training_pages,
#     tokenizer = tokenizer
# )
# print (dataset)

# for batch in dataset:
#     print (batch)
